# The Scaletime Class

The purpose of this class is to provide a means for emulating process control experiments without being rigidly constrained by real time. Scaletime shares the following characteristics with real time:

* monotonically increasing
* continuous
* global 

Relative to real time, scale time can progress at non-uniform rates, faster or slower than real time, and even come to a stop relative to real time.

## Global Characteristics

One of the useful characteristics of real time clocks in software is the ability to coordinate actions among a group of actors without the need for direct messaging.  If we agree on a common epoch, agree to measure time in the same manner, and agree on a schedule of actions, then we can proceed independently. Thus there is a global character to real time.

Here we define a class `Scaletime` for which will need to provide some global information to all instances of its use.  The first piece of information is a common epoch which defines a common begining point for all instances. Here we'll use the time at which the class is defined as the epoch. We'll call the epoch _knowntime.

In [119]:
import time

class Scaletime():
    _known_time = time.time()
    _elapsed_time = 0

# test: instances created at different times should have the same epoch
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert a._known_time == b._known_time

Intervals in Scaletime can proceed faster or slower than real time as determined by scale factor `_scalefactor`. Since all instances of Scaletime need to report that same elapsed time, `_scalefactor` is also class variable.

In [121]:
import time

class Scaletime():
    _known_time = time.time()
    _elapsed_time = 0
    _scalefactor = 1
    
    @classmethod
    def time(cls):
        return cls._elapsed_time + cls._scalefactor*(time.time() - cls._known_time)

# test: instances created at different times should have the same epoch
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert a._known_time == b._known_time
    
# test: instances created at different times should report the same scaled time.
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert abs(b.time() - a.time()) < 0.01

Setting the scale factor is also a class method.

In [122]:
import time

class Scaletime():
    _known_time = time.time()
    _scalefactor = 1
    _elapsed_time = 0
    
    @classmethod
    def time(cls):
        """Return time elapsed since class definition in scaled units."""
        return cls._elapsed_time + cls._scalefactor*(time.time() - cls._known_time)
    
    @classmethod
    def scale(cls, scalefactor):
        """Change the time scale factor, i.e., ratio of scaled time rate to real time."""
        cls._elapsed_time = cls.time()
        cls._known_time = time.time()
        cls._scalefactor = scalefactor


# test: instances created at different times should have the same epoch
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert a._known_time == b._known_time

    
# test: instances created at different times should report the same scaled time.
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert abs(b.time() - a.time()) < 0.01


# test: change scale factors
sf = 2
delay = 1
tic = a.time()
a.scale(sf)
time.sleep(delay)
toc = a.time()
assert abs(toc - tic - sf*delay) < 0.01

A particular instance of the class may be used to implement a sleep to be implemented in scaled time units. Sleep needs to access the class scalefactor, therefore is also implemented as a class method.

In [7]:
import time as time

class Scaletime():
    _known_time = time.time()
    _scalefactor = 1
    _elapsed_time = 0
    _running = True
    
    @classmethod
    def time(cls):
        """Return time elapsed since class definition in scaled units."""
        return cls._elapsed_time + int(cls._running)*cls._scalefactor*(time.time() - cls._known_time)
    
    @classmethod
    def scale(cls, scalefactor):
        """Change the time scale factor, i.e., ratio of scaled time rate to real time."""
        cls._elapsed_time = cls.time()
        cls._known_time = time.time()
        cls._scalefactor = scalefactor
    
    @classmethod
    def sleep(cls, delay):
        """Sleep for a period delay in scaled time units."""
        if cls._scalefactor == 0:
            raise RuntimeError("Can't sleep when scaled clock is stopped.")
        time.sleep(delay/cls._scalefactor)
        
    @classmethod
    def reset(cls):
        """Reset scaled time to zero."""
        cls._known_time = time.time()
        cls._elapsed_time = 0
        
    @classmethod
    def stop(cls):
        """Stop scaled time clock."""
        cls._elapsed_time = cls.time()
        cls._known_time = time.time()
        cls._running = False
        
    @classmethod
    def start(cls):
        """Restart scale time clock using previous scale factor and elapsed time."""
        cls._known_time = time.time()
        cls._running = True
        
        
# test: instances created at different times should have the same epoch
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert a._known_time == b._known_time

    
# test: instances created at different times should report the same scaled time.
a = Scaletime()
time.sleep(2)
b = Scaletime()
assert abs(b.time() - a.time()) < 0.01


# test: change scale factors
sf = 2
delay = 1
a = Scaletime()
tic = a.time()
a.scale(sf)
time.sleep(delay)
toc = a.time()
assert abs(toc - tic - sf*delay) < 0.05


# test: scaled sleep
sf = 5
sdelay = 10
a = Scaletime()
a.scale(sf)
stic = a.time()
tic = time.time()
a.sleep(sdelay)
stoc = a.time()
toc = time.time()
assert abs(stoc - stic - sdelay) < 0.1
assert abs(toc - tic - sdelay/sf) < 0.1


# test: error on zero or negative scale factor

# test: reset to zero
a  = Scaletime()
time.sleep(2)
a.reset()
assert a.time() < 0.01
assert a._elapsed_time == 0

# test: stop
a = Scaletime()
a.stop()
tic = a.time()
time.sleep(1)
toc = a.time()
assert tic==toc
assert a._running == False

# test: start
a = Scaletime()
a.scale(1)
a.stop()
tic = a.time()
atic = a._known_time
time.sleep(2)
a.start()
toc = a.time()
atoc = a._known_time
assert abs(toc - tic) < 0.1
assert abs(atoc - atic - 2) < 0.1
assert a._running == True

In [3]:
scaletime = Scaletime()
scaletime.scale(1)
scaletime.reset()

def run_five(delay):
    for i in range(5):
        time.sleep(delay)
        print(scaletime.time())

print("Real time")
run_five(1)
print("10x")
scaletime.scale(10)
run_five(0.1)
print("Stopped")
scaletime.stop()
run_five(1)
print("Back to real time")
scaletime.scale(1)
scaletime.start()
run_five(1)

Real time
1.0055122375488281
2.010618209838867
3.011207103729248
4.011430263519287
5.011703014373779
10x
6.064152717590332
7.073414325714111
8.078382015228271
9.132602214813232
10.170505046844482
Stopped
10.175094604492188
10.175094604492188
10.175094604492188
10.175094604492188
10.175094604492188
Back to real time
11.1765456199646
12.177032709121704
13.17726182937622
14.178507566452026
15.181276798248291


In [17]:
clocktime = time

def clock(tperiod, tstep=1, tol=0.25):
    """Generator providing time values in sync with real time clock.

    Args:
        tperiod (float): Time interval for clock operation in seconds.
        tstep (float): Time step.
        tol (float): Maximum permissible deviation from real time.

    Yields:
        float: The next time step rounded to nearest 10th of a second.
    """
    clocknow = 0
    clockstart = clocktime.time()
    while clocknow <= tperiod - tstep + tol:
        yield round(clocknow, 1)
        tsleep = tstep - (clocktime.time() - clockstart)% tstep
        clocktime.sleep(tsleep)
        clocknow = (clocktime.time() - clockstart)

    yield round(clocknow, 1)
    
    
realtime = time.time()
#clocktime.scale(4)
for t in clock(5):
    print(t)


0
1.0
2.0
3.0
4.0
5.0


In [80]:
for k in range(0,5):
    print(a.time(), b.time())
    time.sleep(1)

49.49175667762756 49.49176049232483
51.49620461463928 51.4962203502655
53.5081307888031 53.5081684589386
55.513304471969604 55.513322591781616
57.522814989089966 57.52284646034241


# A Scaled Time Class for TCLab

In [9]:
import time as realtime


class Scaledtime(object):
    SPEEDUP = 1
    
    def __init__(self):
        self.tstart = realtime.time()
        self.clocknow = 0

    def time(self):
        return Scaledtime.SPEEDUP*(realtime.time() - self.tstart)
    
    def reset(self, tinitial=0):
        self.tstart = realtime.time() - tinitial/Scaledtime.SPEEDUP

    now = property(fget=time, doc="Current environment time")
    
    def clock(self, tperiod, tstep=1, tol=0.25):
        """Generator providing time values in sync with real time clock.

        Args:
            tperiod (float): Time interval for clock operation in seconds.
            tstep (float): Time step.
            tol (float): Maximum permissible deviation from real time.

        Yields:
            float: The next time step rounded to nearest 10th of a second.
        """
        self.clocknow = 0
        clockstart = realtime.time()
        while self.clocknow <= tperiod - tstep + tol:
            yield round(self.clocknow, 1)
            tsleep = tstep - (Scaledtime.SPEEDUP * (realtime.time() - clockstart))% tstep
            realtime.sleep(max(0, tsleep/Scaledtime.SPEEDUP))
            self.clocknow = Scaledtime.SPEEDUP * (realtime.time() - clockstart)

        yield round(self.clocknow, 1)



In [16]:
from tclab import Clock
import time

clock = Clock()
clock.start()

def run_five(delay):
    for i in range(5):
        time.sleep(delay)
        print(clock.value)

print("Real time")
run_five(1)
print("10x")
clock.speedup = 10
run_five(0.1)
print("Stopped")
clock.stop()
run_five(1)
print("Back to real time")
clock.speedup = 1
clock.start()
run_five(1)


Real time
1.0007102489471436
2.00541615486145
3.005714178085327
4.00611424446106
5.006394147872925
10x
6.054100513458252
7.059900283813477
8.08285903930664
9.100958824157715
10.103070259094238
Stopped
10.10744047164917
10.10744047164917
10.10744047164917
10.10744047164917
10.10744047164917
Back to real time
11.112423419952393
12.112640380859375
13.117499351501465
14.122878313064575
15.12339448928833


In [17]:
from tclab import Scaledtime

a = Scaledtime()
b = Scaledtime()

a.speedup(2)
print(b.SPEEDUP)
b.speedup(3)
print(a.SPEEDUP)

2
3


In [18]:
a.time()

2.958956480026245

In [19]:
b.time()

8.412218570709229

In [22]:
a.tstart

1518805909.053122

## Use as an Iterator

In [10]:
import datetime

stime = Scaledtime()
clock = stime.clock
Scaledtime.SPEEDUP = 1

slbl = "datetime                     clock   clocknow     time()        now"
sfmt = "{0:25s} {1:7.4f}    {2:7.4f}    {3:7.4f}    {4:7.4f}"

print(slbl)
for t in clock(10):
    print(sfmt.format(str(datetime.datetime.now()), t, stime.clocknow, stime.time(), stime.now))

datetime                     clock   clocknow     time()        now
2018-02-15 23:28:06.314883  0.0000     0.0000     0.0004     0.0004
2018-02-15 23:28:07.315549  1.0000     1.0006     1.0011     1.0011
2018-02-15 23:28:08.318834  2.0000     2.0039     2.0044     2.0044
2018-02-15 23:28:09.315280  3.0000     3.0004     3.0008     3.0008
2018-02-15 23:28:10.320030  4.0000     4.0051     4.0056     4.0056
2018-02-15 23:28:11.315079  5.0000     5.0002     5.0006     5.0006
2018-02-15 23:28:12.317682  6.0000     6.0028     6.0032     6.0032
2018-02-15 23:28:13.316153  7.0000     7.0012     7.0017     7.0017
2018-02-15 23:28:14.318771  8.0000     8.0039     8.0043     8.0043
2018-02-15 23:28:15.315955  9.0000     9.0010     9.0015     9.0015
2018-02-15 23:28:16.317242 10.0000    10.0023    10.0028    10.0028


## Scaling Time

The class variable `SPEEDUP` is adjusted to rescale time. Setting `SPEEDUP` rescales time on all class instances.

In [11]:
Scaledtime.SPEEDUP = 10

print(slbl)
for t in clock(10):
    print(sfmt.format(str(datetime.datetime.now()), t, stime.clocknow, stime.time(), stime.now))

datetime                     clock   clocknow     time()        now
2018-02-15 23:28:18.973753  0.0000     0.0000    126.5930    126.5930
2018-02-15 23:28:19.077022  1.0000     1.0324    127.6258    127.6258
2018-02-15 23:28:19.176907  2.0000     2.0312    128.6247    128.6247
2018-02-15 23:28:19.276138  3.0000     3.0236    129.6170    129.6170
2018-02-15 23:28:19.378265  4.0000     4.0448    130.6383    130.6383
2018-02-15 23:28:19.476914  5.0000     5.0313    131.6248    131.6248
2018-02-15 23:28:19.575604  6.0000     6.0182    132.6117    132.6117
2018-02-15 23:28:19.676935  7.0000     7.0315    133.6250    133.6250
2018-02-15 23:28:19.778216  8.0000     8.0444    134.6378    134.6378
2018-02-15 23:28:19.877123  9.0000     9.0334    135.6269    135.6269
2018-02-15 23:28:19.976773 10.0000    10.0299    136.6233    136.6233


Extreme scaling results in some jitter in the generated time steps.

In [12]:
Scaledtime.SPEEDUP = 100

print(slbl)
for t in clock(10):
    print(sfmt.format(str(datetime.datetime.now()), t, stime.clocknow, stime.time(), stime.now))

datetime                     clock   clocknow     time()        now
2018-02-15 23:28:21.385475  0.0000     0.0000    1507.1026    1507.1028
2018-02-15 23:28:21.395575  1.0000     1.0085    1508.1128    1508.1130
2018-02-15 23:28:21.407918  2.2000     2.2419    1509.3470    1509.3471
2018-02-15 23:28:21.415531  3.0000     3.0035    1510.1081    1510.1082
2018-02-15 23:28:21.427290  4.2000     4.1798    1511.2843    1511.2845
2018-02-15 23:28:21.435632  5.0000     5.0131    1512.1191    1512.1194
2018-02-15 23:28:21.447293  6.2000     6.1801    1513.2846    1513.2847
2018-02-15 23:28:21.457324  7.2000     7.1840    1514.2874    1514.2876
2018-02-15 23:28:21.467474  8.2000     8.1996    1515.3027    1515.3028
2018-02-15 23:28:21.476386  9.1000     9.0900    1516.1936    1516.1937
2018-02-15 23:28:21.485698 10.0000    10.0208    1517.1251    1517.1252


## Reset scaled time to an initial value

In [14]:
Scaledtime.SPEEDUP = 2
stime.reset(10)

print(slbl)
for t in clock(10):
    print(sfmt.format(str(datetime.datetime.now()), t, stime.clocknow, stime.time(), stime.now))

datetime                     clock   clocknow     time()        now
2018-02-15 23:29:09.059008  0.0000     0.0000    10.0004    10.0004
2018-02-15 23:29:09.559322  1.0000     1.0006    11.0010    11.0010
2018-02-15 23:29:10.060927  2.0000     2.0038    12.0042    12.0042
2018-02-15 23:29:10.562374  3.0000     3.0067    13.0071    13.0071
2018-02-15 23:29:11.060681  4.0000     4.0033    14.0037    14.0037
2018-02-15 23:29:11.559676  5.0000     5.0013    15.0017    15.0017
2018-02-15 23:29:12.062128  6.0000     6.0062    16.0066    16.0066
2018-02-15 23:29:12.562174  7.0000     7.0063    17.0067    17.0067
2018-02-15 23:29:13.063478  8.0000     8.0089    18.0093    18.0093
2018-02-15 23:29:13.561077  9.0000     9.0040    19.0045    19.0045
2018-02-15 23:29:14.063737 10.0000    10.0094    20.0098    20.0098
